In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

32 documents
10 classes ['buy', 'cake', 'flavors', 'goodbye', 'greeting', 'hours', 'opentoday', 'payments', 'thanks', 'today']
44 unique stemmed words ["'s", 'a', 'acceiv', 'anyon', 'ar', 'avail', 'buy', 'bye', 'cak', 'can', 'card', 'cash', 'credit', 'day', 'do', 'flavo', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'lat', 'mastercard', 'on', 'op', 'see', 'serv', 'tak', 'thank', 'that', 'ther', 'to', 'today', 'want', 'what', 'when', 'wher', 'yo', 'you']


In [11]:
documents

[(['Hi'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'),
 (['What', 'are', 'your', 'hours', '?'], 'hours'),
 (['When', 'are', 'you', 'open', '?'], 'hours'),
 (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'),
 (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'),
 (['Are', 'you', 'cash', 'only', '?'], 'payments'),
 (['Are', 'you', 'open', 'today', '?'], 'opentoday'),
 (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'),
 (['What', 'are', 'your', 'hours', 'today', '?'], 'opentoday'),
 (['today'], 'today'),
 (['Do', 'you', 'serve', 'cakes', '?'], 'cake'),
 (['cakes', '?'], 'cake'),
 (['I', 'want', 'a', 'cake'], 'cake'),
 

In [4]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.config.init_training_mode()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00842 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.00842 - acc: 1.0000 -- iter: 24/32
Training Step: 4000  | total loss: 0.00808 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.00808 - acc: 1.0000 -- iter: 32/32
--
INFO:tensorflow:/home/jovyan/work/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [7]:
p = bow("is the shop open ?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
['buy', 'cake', 'flavors', 'goodbye', 'greeting', 'hours', 'opentoday', 'payments', 'thanks', 'today']


In [8]:
print(model.predict([p]))

[[  6.66965441e-11   3.85853056e-08   4.21989173e-01   3.21600351e-06
    3.13728333e-06   1.86921120e-01   2.57882141e-02   1.94774311e-05
    3.65242630e-01   3.30518451e-05]]


In [9]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )